In [14]:
!pip -q install torch-geometric


In [15]:
import os, json, time, random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj

# For reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = {
    "dataset_root": "./data",
    "dataset_name": "Cora",
    "hidden_dim": 16,
    "dropout_p": 0.5,
    "lr": 0.01,
    "weight_decay": 5e-4,
    "epochs": 200,
    "patience": 50,
}


In [16]:
dataset = Planetoid(root=config["dataset_root"], name=config["dataset_name"], transform=T.NormalizeFeatures())
data = dataset[0].to(device)

x = data.x
y = data.y
train_mask = data.train_mask
val_mask = data.val_mask
test_mask = data.test_mask

print(f"Dataset: {dataset}")
print(f"Nodes: {data.num_nodes}, Edges: {data.num_edges}, Features: {data.num_node_features}, Classes: {dataset.num_classes}")


Dataset: Cora()
Nodes: 2708, Edges: 10556, Features: 1433, Classes: 7


In [17]:
adj_dense = to_dense_adj(data.edge_index, max_num_nodes=data.num_nodes)[0].to(device)

def renormalize_adj(adj: torch.Tensor) -> torch.Tensor:
    # Â = A + I
    adj_tilde = adj + torch.eye(adj.size(0), device=adj.device, dtype=adj.dtype)
    # D^{-1/2} Â D^{-1/2}
    deg = adj_tilde.sum(dim=1)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[torch.isinf(deg_inv_sqrt)] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj_tilde @ D_inv_sqrt

adj_norm = renormalize_adj(adj_dense)


In [18]:
class GCNLayer(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)

    def forward(self, adj_norm: torch.Tensor, h: torch.Tensor) -> torch.Tensor:
        return self.linear(adj_norm @ h)

class GCN(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int, dropout_p: float = 0.5):
        super().__init__()
        self.gcn1 = GCNLayer(in_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, out_dim)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, x: torch.Tensor, adj_norm: torch.Tensor) -> torch.Tensor:
        h = self.gcn1(adj_norm, x)
        h = F.relu(h)
        h = self.dropout(h)
        logits = self.gcn2(adj_norm, h)
        return logits

class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, out_dim: int, dropout_p: float = 0.5):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = F.relu(self.fc1(x))
        h = self.dropout(h)
        logits = self.fc2(h)
        return logits


In [19]:
@torch.no_grad()
def accuracy_from_logits(logits: torch.Tensor, y_true: torch.Tensor) -> float:
    preds = logits.argmax(dim=1)
    return float((preds == y_true).float().mean().item())

@torch.no_grad()
def evaluate_gcn(model: nn.Module, mask: torch.Tensor, criterion) -> dict:
    model.eval()
    logits = model(x, adj_norm)
    loss = float(criterion(logits[mask], y[mask]).item())
    acc = accuracy_from_logits(logits[mask], y[mask])
    return {"loss": loss, "acc": acc}

@torch.no_grad()
def evaluate_mlp(model: nn.Module, mask: torch.Tensor, criterion) -> dict:
    model.eval()
    logits = model(x)
    loss = float(criterion(logits[mask], y[mask]).item())
    acc = accuracy_from_logits(logits[mask], y[mask])
    return {"loss": loss, "acc": acc}

def train_one_run(model_name: str, seed: int, cfg: dict) -> dict:
    set_seed(seed)

    if model_name == "GCN":
        model = GCN(in_dim=x.size(1), hidden_dim=cfg["hidden_dim"], out_dim=dataset.num_classes, dropout_p=cfg["dropout_p"]).to(device)
        eval_fn = evaluate_gcn
        forward_train = lambda m: m(x, adj_norm)
    elif model_name == "MLP":
        model = MLP(in_dim=x.size(1), hidden_dim=cfg["hidden_dim"], out_dim=dataset.num_classes, dropout_p=cfg["dropout_p"]).to(device)
        eval_fn = evaluate_mlp
        forward_train = lambda m: m(x)
    else:
        print("Hehehe")

    optimizer = torch.optim.Adam(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    criterion = nn.CrossEntropyLoss()

    best = {"val_loss": float("inf"), "val_acc": -1.0, "epoch": -1, "state": None}
    patience = cfg.get("patience", None)
    so_lan_doi = 0

    history = {"train_loss": [], "val_loss": [], "val_acc": []}
    t0 = time.time()

    pbar = tqdm(range(1, cfg["epochs"] + 1), desc=f"{model_name} seed={seed}", leave=False)
    for epoch in pbar:
        model.train()
        optimizer.zero_grad()

        logits = forward_train(model)
        loss = criterion(logits[train_mask], y[train_mask])
        loss.backward()
        optimizer.step()

        history["train_loss"].append(float(loss.item()))

        val_metrics = eval_fn(model, val_mask, criterion)
        history["val_loss"].append(val_metrics["loss"])
        history["val_acc"].append(val_metrics["acc"])

        pbar.set_postfix(
            tr_loss=f"{loss.item():.4f}",
            val_loss=f"{val_metrics['loss']:.4f}",
            val_acc=f"{val_metrics['acc']:.4f}"
        )

        if val_metrics["loss"] < best["val_loss"]:
            best["val_loss"] = val_metrics["loss"]
            best["val_acc"] = val_metrics["acc"]
            best["epoch"] = epoch
            best["state"] = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            so_lan_doi = 0
        else:
            if patience is not None:
                so_lan_doi += 1
                if so_lan_doi >= patience:
                    break
    pbar.close()

    # load best n eval test
    model.load_state_dict({k: v.to(device) for k, v in best["state"].items()})
    if model_name == "GCN":
        test_logits = model(x, adj_norm)
    else:
        test_logits = model(x)
    test_loss = float(criterion(test_logits[test_mask], y[test_mask]).item())
    test_acc = accuracy_from_logits(test_logits[test_mask], y[test_mask])

    elapsed = time.time() - t0

    return {
        "model": model_name,
        "seed": seed,
        "best_epoch": best["epoch"],
        "best_val_loss": best["val_loss"],
        "best_val_acc": best["val_acc"],
        "best_test_loss": test_loss,
        "best_test_acc": test_acc,
        "elapsed_sec": elapsed,
        "config": cfg,
        "history": history,
    }


In [20]:
#run 3 lan voi 3 seeds khac nhau
seeds = [0, 1, 2]

all_results = []
for s in seeds:
    all_results.append(train_one_run("GCN", seed=s, cfg=config))
for s in seeds:
    all_results.append(train_one_run("MLP", seed=s, cfg=config))

df = pd.DataFrame(all_results)
metrics = ["best_val_loss","best_val_acc","best_test_loss","best_test_acc"]
agg = df.groupby("model")[metrics].agg(["mean","std","count"]).reset_index()

def format_number(mu, sd, digits=4):
    return f"{mu:.{digits}f} ± {sd:.{digits}f}"

pretty = pd.DataFrame({"model": agg["model"]})
for m in metrics:
    pretty[m] = [format_number(mu, (sd if not np.isnan(sd) else 0.0)) for mu, sd in zip(agg[(m,"mean")], agg[(m,"std")])]

display(pretty)

pretty.to_csv("results_summary_mean_std.csv", index=False)
with open("results_all_runs.json","w") as f:
    json.dump({"seeds": seeds, "results": all_results, "config": config}, f, indent=2)


GCN seed=0:   0%|          | 0/200 [00:00<?, ?it/s]

GCN seed=1:   0%|          | 0/200 [00:00<?, ?it/s]

GCN seed=2:   0%|          | 0/200 [00:00<?, ?it/s]

MLP seed=0:   0%|          | 0/200 [00:00<?, ?it/s]

MLP seed=1:   0%|          | 0/200 [00:00<?, ?it/s]

MLP seed=2:   0%|          | 0/200 [00:00<?, ?it/s]

,model,best_val_loss,best_val_acc,best_test_loss,best_test_acc
0,GCN,0.8435 ± 0.0121,0.7907 ± 0.0031,0.8027 ± 0.0080,0.8077 ± 0.0025
1,MLP,1.3657 ± 0.0496,0.5773 ± 0.0175,1.3332 ± 0.0391,0.5770 ± 0.0144
